https://github.com/MorvanZhou/pytorch-A3C

위 링크보고 따라해보는 페이지..

DQN해본다음에 해보는거.. DQN은 gym-dqn 이 문서보면 내가 따라해본게 기록돼 있다.

음 좀 공부해보니까.. a3c가 아니라 actor critic까지만 구현해놓은거 같은데.. 좀 더 들여다 보자

아무래도 actor critic에다가 aync 까지만 구현해놓은것 같다 ㅋㅋ

In [1]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import gym
import os
os.environ["OMP_NUM_THREADS"] = "1"

UPDATE_GLOBAL_ITER = 10
GAMMA = 0.9
MAX_EP = 10000

env = gym.make('CartPole-v0')

# 오호.. 아래처럼 space수(?)랑 action수를 뽑아올 수 있구만
N_S = env.observation_space.shape[0]
N_A = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [2]:
# 레이어 별로 아래처럼 초기화 하는거 쉽고 재밌네 ㅎ
def set_init(layers):
    for layer in layers:
        nn.init.normal_(layer.weight, mean=0., std=0.1)
        nn.init.constant_(layer.bias, 0.1)

# 지금까지 봤던 pytorch net과 다르게 choose_action()이라는 메소드와 loss_func()라는 메소드가 보이네..
# 음? 그리고 CNN이 아닌듯하다?? conv가 안보이네?
# 인풋이 픽셀이 아니겠구만.. 살펴봐야겠네..
class Net(nn.Module):
    def __init__(self, s_dim, a_dim):
        super(Net, self).__init__()
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.pi1 = nn.Linear(s_dim, 100)
        self.pi2 = nn.Linear(100, a_dim)
        self.v1 = nn.Linear(s_dim, 100)
        self.v2 = nn.Linear(100, 1)
        set_init([self.pi1, self.pi2, self.v1, self.v2])
        self.distribution = torch.distributions.Categorical

    # 굉장히 골때리는 걸 하나 발견했다.
    # 아래 x가 원래는 [ 0.0490, -0.0435, -0.0430, -0.0198] 이런식으로 4개 노드 넣는건데..
    # 아래 처럼 N개를 한꺼번에 넣는게 허용이되네?? 이러면 아웃풋도 여러개를 뱉어내는건지..
    ''' tensor([[-0.0288,  0.0320,  0.0333,  0.0161],
        [-0.0282, -0.1636,  0.0336,  0.3191],
        ...
        [-0.0235,  0.6108,  0.0463, -0.7183]])
        '''
    # 그러네.. 추적해보니 여러개를 배열로 뱉어낸다.. 신기~
    def forward(self, x):
        pi1 = F.relu(self.pi1(x))
        logits = self.pi2(pi1)
        v1 = F.relu(self.v1(x))
        values = self.v2(v1)
        return logits, values

    def choose_action(self, s):
        self.eval()
        logits, _ = self.forward(s)
        prob = F.softmax(logits, dim=1).data
        m = self.distribution(prob)
        # 아래 sample이란거 좀 재밌네.. distribution의 확률대로 인덱스 하나 픽해서 리턴해주는거 같다.
        # 예를 들어 Categorical의 distribution이 [0.4, 0.6] 이라고 한다면
        # 0.4의 확률로 0이 나오고 0.6의 확률로 1이 나오는듯
        # 즉, value gradient 방식인 DQN과 다르게 policy gradient방식을 사용한다는 이야기
        return m.sample().numpy()[0]

    # s는 최근 10번동안의 gym observation들..observation하나는 [ 0.0490, -0.0435, -0.0430, -0.0198] 이런식으로 숫자 4개로 구성
    # a는 최근 10번동안 action list.. [0, 1, 1, 0, 0, 0, 0, 1, 0, 1] 이런식
    def loss_func(self, s, a, v_t):
        self.train()
        logits, values = self.forward(s)
        td = v_t - values
        c_loss = td.pow(2)

        probs = F.softmax(logits, dim=1)
        m = self.distribution(probs)
        exp_v = m.log_prob(a) * td.detach().squeeze()
        a_loss = -exp_v
        total_loss = (c_loss + a_loss).mean()
        return total_loss

In [3]:
def v_wrap(np_array, dtype=np.float32):
    if np_array.dtype != dtype:
       np_array = np_array.astype(dtype)
    return torch.from_numpy(np_array)

def push_and_pull(opt, lnet, gnet, done, s_, bs, ba, br, gamma):
    if done:
        v_s_ = 0.               # terminal
    else:
        v_s_ = lnet.forward(v_wrap(s_[None, :]))[-1].data.numpy()[0, 0]

    buffer_v_target = []
    for r in br[::-1]:    # reverse buffer r
        v_s_ = r + gamma * v_s_
        buffer_v_target.append(v_s_)
    buffer_v_target.reverse()

    loss = lnet.loss_func(
        v_wrap(np.vstack(bs)),
        v_wrap(np.array(ba), dtype=np.int64) if ba[0].dtype == np.int64 else v_wrap(np.vstack(ba)),
        v_wrap(np.array(buffer_v_target)[:, None]))

    # 뭔가 아래가 핵심인것 같다. 
    # 로컬 gradient를 계산해서 global로 업데이트 쳐주는 내용인것 같은데..
    # 업데이트 후에 opt.step()을 하네? 이러면 step()의 영향력은 local에만 있지 않나?...
    # 아니네 opt는 gnet에 매핑되어 있네!!
    # calculate local gradients and push local parameters to global
    opt.zero_grad()
    loss.backward()
    for lp, gp in zip(lnet.parameters(), gnet.parameters()):
        gp._grad = lp.grad
    opt.step()

    # pull global parameters
    lnet.load_state_dict(gnet.state_dict())

def record(global_ep, global_ep_r, ep_r, res_queue, name):
    with global_ep.get_lock():
        global_ep.value += 1
    with global_ep_r.get_lock():
        if global_ep_r.value == 0.:
            global_ep_r.value = ep_r
        else:
            global_ep_r.value = global_ep_r.value * 0.99 + ep_r * 0.01
    res_queue.put(global_ep_r.value)
    print(
        name,
        "Ep:", global_ep.value,
        "| Ep_r: %.0f" % global_ep_r.value,
    )
    
class Worker(mp.Process):
    def __init__(self, gnet, opt, global_ep, global_ep_r, res_queue, name):
        super(Worker, self).__init__()
        self.name = 'w%i' % name
        self.g_ep, self.g_ep_r, self.res_queue = global_ep, global_ep_r, res_queue
        self.gnet, self.opt = gnet, opt
        #local 네트웍을 따로 가지고 있네? gloal하고 서로 카피하거나 업데이트 하는 과정은 어떻게 되지?
        self.lnet = Net(N_S, N_A)           # local network
        self.env = gym.make('CartPole-v0').unwrapped

    def run(self):
        total_step = 1
        while self.g_ep.value < MAX_EP:
            # s에는 initial observation이 return되고 의미는 아래와 같다.
            # [position of cart, velocity of cart, angle of pole, rotation rate of pole]. 
            # Defined at https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py#L75
            # 당연히 호출시마다 리턴값은 계속 랜덤하게 바뀐다.
            s = self.env.reset()
            buffer_s, buffer_a, buffer_r = [], [], []
            ep_r = 0.
            while True:
                # 아.. 화면에는 0번 프로세스만 표시하나 보네..
                if self.name == 'w0':
                    self.env.render()
                a = self.lnet.choose_action(v_wrap(s[None, :]))
                s_, r, done, _ = self.env.step(a)
                if done: r = -1
                ep_r += r
                buffer_a.append(a)
                buffer_s.append(s)
                buffer_r.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0 or done:  # update global and assign to local net
                    # sync
                    push_and_pull(self.opt, self.lnet, self.gnet, done, s_, buffer_s, buffer_a, buffer_r, GAMMA)
                    buffer_s, buffer_a, buffer_r = [], [], []

                    if done:  # done and print information
                        record(self.g_ep, self.g_ep_r, ep_r, self.res_queue, self.name)
                        break
                s = s_
                total_step += 1
        self.res_queue.put(None)

In [9]:
# 이거는 optimizer에 대해서 tensor들 shared_memory()호출해주는게 핵심이 아닐까 싶은데 exp_avg 이런게 먼지 모르겠고
# 정확하게 분석은 해보기 전이다. 나중에 해보자
class SharedAdam(torch.optim.Adam):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.9), eps=1e-8,
                 weight_decay=0):
        super(SharedAdam, self).__init__(params, lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        # State initialization
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['exp_avg'] = torch.zeros_like(p.data)
                state['exp_avg_sq'] = torch.zeros_like(p.data)

                # share in memory
                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

In [20]:
if __name__ == "__main__":
    gnet = Net(N_S, N_A)        # global network
    gnet.share_memory()         # share the global parameters in multiprocessing
    opt = SharedAdam(gnet.parameters(), lr=0.0001)      # global optimizer
    # mp간 서로 공유되는 변수랑 queue를 아래처럼 선언할 수 있는걸로 보이네? 굳~
    # 아마도 lock처리가 알아서 되겠지? 봐뒀다가 나중에 써보자
    global_ep, global_ep_r, res_queue = mp.Value('i', 0), mp.Value('d', 0.), mp.Queue()

    # parallel training
    workers = [Worker(gnet, opt, global_ep, global_ep_r, res_queue, i) for i in range(mp.cpu_count())]
    [w.start() for w in workers]
    res = []                    # record episode reward to plot

    while True:
        r = res_queue.get()
        if r is not None:
            res.append(r)
        else:
            break
    [w.join() for w in workers]



WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
w1 Ep: 2 | Ep_r: 12
w3 Ep: 2 | Ep_r: 12
w2 Ep: 3 | Ep_r: 12
w3 Ep: 4 | Ep_r: 12
w2 Ep: 5 | Ep_r: 12
w1 Ep: 6 | Ep_r: 12
w2 Ep: 7 | Ep_r: 12
w3 Ep: 9 | Ep_r: 13
w1 Ep: 8 | Ep_r: 12
w1 Ep: 10 | Ep_r: 13
w2 Ep: 11 | Ep_r: 13
w3 Ep: 12 | Ep_r: 13
w3 Ep: 13 | Ep_r: 13
w3 Ep: 14 | Ep_r: 13
w3 Ep: 15 | Ep_r: 13
w2 Ep: 16 | Ep_r: 13
w3 Ep: 17 | Ep_r: 13
w3 Ep: 18 | Ep_r: 13
w3 Ep: 19 | Ep_r: 13
w3 Ep: 20 | Ep_r: 13
w3 Ep: 21 | Ep_r: 13
w3 Ep: 22 | Ep_r: 13
w3 Ep: 24 | Ep_r: 14
w2 Ep: 23 | Ep_r: 14
w3 Ep: 25 | Ep_r: 14
w3 Ep: 26 | Ep_r: 14
w3 Ep: 27 | Ep_r: 14
w2 Ep: 28 | Ep_r: 14
w1 Ep: 29 | Ep_r: 14
w1 E

w1 Ep: 359 | Ep_r: 22
w1 Ep: 360 | Ep_r: 22
w1 Ep: 361 | Ep_r: 22
w3 Ep: 362 | Ep_r: 22
w1 Ep: 363 | Ep_r: 22
w0 Ep: 364 | Ep_r: 22
w1 Ep: 365 | Ep_r: 22
w1 Ep: 366 | Ep_r: 22
w1 Ep: 367 | Ep_r: 22
w2 Ep: 368 | Ep_r: 22
w1 Ep: 369 | Ep_r: 22
w1 Ep: 370 | Ep_r: 22
w1 Ep: 371 | Ep_r: 22
w3 Ep: 372 | Ep_r: 22
w2 Ep: 373 | Ep_r: 22
w3 Ep: 374 | Ep_r: 22
w2 Ep: 375 | Ep_r: 22
w3 Ep: 376 | Ep_r: 22
w2 Ep: 377 | Ep_r: 22
w3 Ep: 378 | Ep_r: 22
w2 Ep: 379 | Ep_r: 22
w3 Ep: 380 | Ep_r: 22
w3 Ep: 381 | Ep_r: 21
w2 Ep: 382 | Ep_r: 22
w3 Ep: 383 | Ep_r: 21
w3 Ep: 384 | Ep_r: 21
w2 Ep: 385 | Ep_r: 22
w3 Ep: 386 | Ep_r: 22
w2 Ep: 387 | Ep_r: 21
w3 Ep: 388 | Ep_r: 21
w2 Ep: 389 | Ep_r: 21
w3 Ep: 390 | Ep_r: 21
w2 Ep: 391 | Ep_r: 21
w3 Ep: 392 | Ep_r: 21
w2 Ep: 393 | Ep_r: 21
w2 Ep: 394 | Ep_r: 21
w1 Ep: 395 | Ep_r: 22
w1 Ep: 396 | Ep_r: 22
w1 Ep: 397 | Ep_r: 22
w2 Ep: 398 | Ep_r: 22
w3 Ep: 399 | Ep_r: 22
w1 Ep: 400 | Ep_r: 22
w2 Ep: 401 | Ep_r: 22
w3 Ep: 402 | Ep_r: 22
w3 Ep: 403 | Ep_r: 22
w1 Ep: 404

w3 Ep: 732 | Ep_r: 34
w3 Ep: 733 | Ep_r: 34
w1 Ep: 734 | Ep_r: 34
w3 Ep: 735 | Ep_r: 34
w0 Ep: 736 | Ep_r: 33
w1 Ep: 738 | Ep_r: 34
w2 Ep: 738 | Ep_r: 34
w3 Ep: 739 | Ep_r: 33
w2 Ep: 740 | Ep_r: 33
w3 Ep: 741 | Ep_r: 33
w2 Ep: 742 | Ep_r: 33
w1 Ep: 743 | Ep_r: 34
w2 Ep: 744 | Ep_r: 33
w3 Ep: 745 | Ep_r: 33
w2 Ep: 746 | Ep_r: 33
w2 Ep: 747 | Ep_r: 33
w2 Ep: 748 | Ep_r: 33
w2 Ep: 749 | Ep_r: 33
w3 Ep: 750 | Ep_r: 33
w2 Ep: 751 | Ep_r: 33
w2 Ep: 753 | Ep_r: 33
w1 Ep: 752 | Ep_r: 33
w3 Ep: 754 | Ep_r: 33
w2 Ep: 755 | Ep_r: 33
w3 Ep: 756 | Ep_r: 33
w3 Ep: 757 | Ep_r: 32
w1 Ep: 758 | Ep_r: 33
w1 Ep: 759 | Ep_r: 33
w3 Ep: 760 | Ep_r: 33
w1 Ep: 761 | Ep_r: 33
w1 Ep: 762 | Ep_r: 33
w1 Ep: 763 | Ep_r: 33
w3 Ep: 764 | Ep_r: 33
w2 Ep: 765 | Ep_r: 34
w1 Ep: 766 | Ep_r: 33
w3 Ep: 767 | Ep_r: 33
w1 Ep: 768 | Ep_r: 33
w2 Ep: 769 | Ep_r: 33
w3 Ep: 770 | Ep_r: 33
w2 Ep: 771 | Ep_r: 33
w1 Ep: 772 | Ep_r: 33
w2 Ep: 773 | Ep_r: 33
w1 Ep: 774 | Ep_r: 33
w2 Ep: 775 | Ep_r: 33
w3 Ep: 776 | Ep_r: 33
w2 Ep: 777

w1 Ep: 1100 | Ep_r: 43
w3 Ep: 1101 | Ep_r: 43
w2 Ep: 1102 | Ep_r: 43
w3 Ep: 1103 | Ep_r: 43
w3 Ep: 1104 | Ep_r: 43
w2 Ep: 1105 | Ep_r: 43
w2 Ep: 1106 | Ep_r: 43
w3 Ep: 1107 | Ep_r: 43
w1 Ep: 1108 | Ep_r: 43
w2 Ep: 1109 | Ep_r: 43
w1 Ep: 1110 | Ep_r: 42
w2 Ep: 1111 | Ep_r: 42
w3 Ep: 1112 | Ep_r: 43
w3 Ep: 1113 | Ep_r: 42
w2 Ep: 1114 | Ep_r: 42
w1 Ep: 1115 | Ep_r: 42
w3 Ep: 1116 | Ep_r: 42
w2 Ep: 1117 | Ep_r: 42
w3 Ep: 1118 | Ep_r: 42
w1 Ep: 1119 | Ep_r: 42
w3 Ep: 1120 | Ep_r: 42
w2 Ep: 1121 | Ep_r: 42
w1 Ep: 1122 | Ep_r: 42
w2 Ep: 1123 | Ep_r: 42
w3 Ep: 1124 | Ep_r: 42
w3 Ep: 1125 | Ep_r: 42
w3 Ep: 1126 | Ep_r: 41
w2 Ep: 1127 | Ep_r: 42
w1 Ep: 1128 | Ep_r: 42
w3 Ep: 1129 | Ep_r: 42
w1 Ep: 1130 | Ep_r: 42
w2 Ep: 1131 | Ep_r: 42
w3 Ep: 1132 | Ep_r: 41
w1 Ep: 1133 | Ep_r: 42
w2 Ep: 1134 | Ep_r: 42
w3 Ep: 1135 | Ep_r: 42
w1 Ep: 1136 | Ep_r: 42
w2 Ep: 1137 | Ep_r: 42
w1 Ep: 1138 | Ep_r: 42
w3 Ep: 1139 | Ep_r: 42
w1 Ep: 1140 | Ep_r: 42
w3 Ep: 1141 | Ep_r: 42
w1 Ep: 1142 | Ep_r: 42
w2 Ep: 1143

w2 Ep: 1457 | Ep_r: 59
w1 Ep: 1458 | Ep_r: 59
w3 Ep: 1459 | Ep_r: 60
w1 Ep: 1460 | Ep_r: 60
w2 Ep: 1461 | Ep_r: 62
w2 Ep: 1462 | Ep_r: 61
w1 Ep: 1463 | Ep_r: 62
w3 Ep: 1464 | Ep_r: 62
w1 Ep: 1465 | Ep_r: 62
w2 Ep: 1466 | Ep_r: 63
w2 Ep: 1467 | Ep_r: 62
w3 Ep: 1468 | Ep_r: 63
w1 Ep: 1469 | Ep_r: 63
w3 Ep: 1470 | Ep_r: 63
w3 Ep: 1471 | Ep_r: 62
w2 Ep: 1472 | Ep_r: 63
w3 Ep: 1473 | Ep_r: 63
w3 Ep: 1474 | Ep_r: 62
w3 Ep: 1475 | Ep_r: 62
w1 Ep: 1476 | Ep_r: 62
w3 Ep: 1477 | Ep_r: 62
w2 Ep: 1478 | Ep_r: 63
w1 Ep: 1479 | Ep_r: 63
w3 Ep: 1480 | Ep_r: 63
w1 Ep: 1481 | Ep_r: 63
w2 Ep: 1482 | Ep_r: 63
w3 Ep: 1483 | Ep_r: 62
w2 Ep: 1484 | Ep_r: 63
w2 Ep: 1485 | Ep_r: 63
w1 Ep: 1486 | Ep_r: 64
w2 Ep: 1488 | Ep_r: 64
w3 Ep: 1488 | Ep_r: 64
w1 Ep: 1489 | Ep_r: 65
w2 Ep: 1490 | Ep_r: 65
w1 Ep: 1491 | Ep_r: 65
w3 Ep: 1492 | Ep_r: 65
w2 Ep: 1493 | Ep_r: 64
w1 Ep: 1494 | Ep_r: 64
w2 Ep: 1495 | Ep_r: 64
w1 Ep: 1496 | Ep_r: 64
w0 Ep: 1497 | Ep_r: 64
w1 Ep: 1499 | Ep_r: 63
w3 Ep: 1499 | Ep_r: 63
w1 Ep: 1500

...

w2 Ep: 8387 | Ep_r: 7
w0 Ep: 8386 | Ep_r: 7
w1 Ep: 8389 | Ep_r: 7
w2 Ep: 8390 | Ep_r: 7
w3 Ep: 8388 | Ep_r: 7
w1 Ep: 8391 | Ep_r: 7
w2 Ep: 8392 | Ep_r: 7
w1 Ep: 8393 | Ep_r: 7
w2 Ep: 8394 | Ep_r: 7
w2 Ep: 8395 | Ep_r: 7
w1 Ep: 8396 | Ep_r: 7
w3 Ep: 8397 | Ep_r: 7
w2 Ep: 8398 | Ep_r: 7
w0 Ep: 8399 | Ep_r: 7
w3 Ep: 8400 | Ep_r: 7
w2 Ep: 8401 | Ep_r: 7
w3 Ep: 8402 | Ep_r: 7
w0 Ep: 8403 | Ep_r: 7
w3 Ep: 8404 | Ep_r: 7
w2 Ep: 8405 | Ep_r: 7
w3 Ep: 8406 | Ep_r: 7
w0 Ep: 8407 | Ep_r: 7
w2 Ep: 8408 | Ep_r: 7
w3 Ep: 8409 | Ep_r: 7
w3 Ep: 8410 | Ep_r: 7
w0 Ep: 8411 | Ep_r: 7
w2 Ep: 8412 | Ep_r: 7
w3 Ep: 8413 | Ep_r: 7
w2 Ep: 8414 | Ep_r: 7
w3 Ep: 8415 | Ep_r: 7
w0 Ep: 8416 | Ep_r: 7
w3 Ep: 8417 | Ep_r: 7
w2 Ep: 8418 | Ep_r: 7
w3 Ep: 8419 | Ep_r: 7
w3 Ep: 8420 | Ep_r: 7
w2 Ep: 8422 | Ep_r: 7
w0 Ep: 8423 | Ep_r: 7
w3 Ep: 8424 | Ep_r: 7
w2 Ep: 8425 | Ep_r: 7
w3 Ep: 8426 | Ep_r: 7
w0 Ep: 8427 | Ep_r: 7
w1 Ep: 8421 | Ep_r: 7
w2 Ep: 8428 | Ep_r: 7
w3 Ep: 8429 | Ep_r: 7
w3 Ep: 8430 | Ep_r: 7
w2 Ep: 843

w1 Ep: 8759 | Ep_r: 7
w2 Ep: 8760 | Ep_r: 7
w3 Ep: 8762 | Ep_r: 7
w1 Ep: 8762 | Ep_r: 7
w2 Ep: 8763 | Ep_r: 7
w2 Ep: 8764 | Ep_r: 7
w3 Ep: 8765 | Ep_r: 7
w2 Ep: 8766 | Ep_r: 7
w1 Ep: 8767 | Ep_r: 7
w3 Ep: 8768 | Ep_r: 7
w2 Ep: 8769 | Ep_r: 7
w1 Ep: 8770 | Ep_r: 7
w2 Ep: 8771 | Ep_r: 7
w3 Ep: 8772 | Ep_r: 7
w2 Ep: 8773 | Ep_r: 7
w1 Ep: 8774 | Ep_r: 7
w2 Ep: 8775 | Ep_r: 7
w3 Ep: 8776 | Ep_r: 7
w1 Ep: 8777 | Ep_r: 7
w2 Ep: 8778 | Ep_r: 7
w1 Ep: 8779 | Ep_r: 7
w2 Ep: 8780 | Ep_r: 7
w3 Ep: 8781 | Ep_r: 7
w2 Ep: 8782 | Ep_r: 7
w3 Ep: 8783 | Ep_r: 7
w3 Ep: 8786 | Ep_r: 7
w2 Ep: 8785 | Ep_r: 7
w1 Ep: 8784 | Ep_r: 7
w0 Ep: 8787 | Ep_r: 7
w1 Ep: 8788 | Ep_r: 7
w3 Ep: 8789 | Ep_r: 7
w0 Ep: 8790 | Ep_r: 7
w3 Ep: 8791 | Ep_r: 7
w1 Ep: 8792 | Ep_r: 7
w3 Ep: 8793 | Ep_r: 7
w1 Ep: 8794 | Ep_r: 7
w0 Ep: 8795 | Ep_r: 7
w1 Ep: 8796 | Ep_r: 7
w3 Ep: 8797 | Ep_r: 7
w1 Ep: 8798 | Ep_r: 7
w3 Ep: 8799 | Ep_r: 7
w0 Ep: 8800 | Ep_r: 7
w1 Ep: 8801 | Ep_r: 7
w3 Ep: 8802 | Ep_r: 7
w0 Ep: 8803 | Ep_r: 7
w3 Ep: 880

w3 Ep: 9132 | Ep_r: 7
w1 Ep: 9133 | Ep_r: 7
w3 Ep: 9134 | Ep_r: 7
w1 Ep: 9136 | Ep_r: 7
w3 Ep: 9136 | Ep_r: 7
w3 Ep: 9138 | Ep_r: 7
w2 Ep: 9137 | Ep_r: 7
w1 Ep: 9139 | Ep_r: 7
w3 Ep: 9140 | Ep_r: 7
w1 Ep: 9141 | Ep_r: 7
w3 Ep: 9142 | Ep_r: 7
w1 Ep: 9143 | Ep_r: 7
w3 Ep: 9144 | Ep_r: 7
w0 Ep: 9145 | Ep_r: 7
w3 Ep: 9147 | Ep_r: 7
w1 Ep: 9146 | Ep_r: 7
w3 Ep: 9148 | Ep_r: 7
w1 Ep: 9149 | Ep_r: 7
w2 Ep: 9150 | Ep_r: 7
w3 Ep: 9151 | Ep_r: 7
w1 Ep: 9152 | Ep_r: 7
w3 Ep: 9153 | Ep_r: 7
w1 Ep: 9154 | Ep_r: 7
w3 Ep: 9155 | Ep_r: 7
w2 Ep: 9156 | Ep_r: 7
w3 Ep: 9157 | Ep_r: 7
w1 Ep: 9158 | Ep_r: 7
w3 Ep: 9159 | Ep_r: 7
w1 Ep: 9161 | Ep_r: 7
w0 Ep: 9160 | Ep_r: 7
w3 Ep: 9162 | Ep_r: 7
w1 Ep: 9163 | Ep_r: 7
w3 Ep: 9164 | Ep_r: 7
w2 Ep: 9165 | Ep_r: 7
w1 Ep: 9166 | Ep_r: 7
w3 Ep: 9167 | Ep_r: 7
w3 Ep: 9169 | Ep_r: 7
w2 Ep: 9168 | Ep_r: 7
w1 Ep: 9170 | Ep_r: 7
w0 Ep: 9171 | Ep_r: 7
w2 Ep: 9172 | Ep_r: 7
w2 Ep: 9173 | Ep_r: 7
w0 Ep: 9174 | Ep_r: 7
w2 Ep: 9177 | Ep_r: 7
w3 Ep: 9177 | Ep_r: 7
w2 Ep: 917

w1 Ep: 9506 | Ep_r: 7
w0 Ep: 9507 | Ep_r: 7
w2 Ep: 9508 | Ep_r: 7
w3 Ep: 9506 | Ep_r: 7
w1 Ep: 9510 | Ep_r: 7
w0 Ep: 9510 | Ep_r: 7
w1 Ep: 9511 | Ep_r: 7
w2 Ep: 9512 | Ep_r: 7
w0 Ep: 9513 | Ep_r: 7
w3 Ep: 9514 | Ep_r: 7
w1 Ep: 9515 | Ep_r: 7
w2 Ep: 9516 | Ep_r: 7
w3 Ep: 9517 | Ep_r: 7
w2 Ep: 9518 | Ep_r: 7
w3 Ep: 9519 | Ep_r: 7
w2 Ep: 9520 | Ep_r: 7
w3 Ep: 9521 | Ep_r: 7
w2 Ep: 9522 | Ep_r: 7
w3 Ep: 9523 | Ep_r: 7
w2 Ep: 9524 | Ep_r: 7
w3 Ep: 9526 | Ep_r: 7
w1 Ep: 9525 | Ep_r: 7
w2 Ep: 9527 | Ep_r: 7
w3 Ep: 9528 | Ep_r: 7
w1 Ep: 9529 | Ep_r: 7
w2 Ep: 9530 | Ep_r: 7
w3 Ep: 9531 | Ep_r: 7
w1 Ep: 9532 | Ep_r: 7
w2 Ep: 9533 | Ep_r: 7
w3 Ep: 9534 | Ep_r: 7
w2 Ep: 9535 | Ep_r: 7
w1 Ep: 9536 | Ep_r: 7
w3 Ep: 9537 | Ep_r: 7
w2 Ep: 9539 | Ep_r: 7
w3 Ep: 9538 | Ep_r: 7
w1 Ep: 9540 | Ep_r: 7
w3 Ep: 9541 | Ep_r: 7
w2 Ep: 9542 | Ep_r: 7
w3 Ep: 9543 | Ep_r: 7
w1 Ep: 9544 | Ep_r: 7
w2 Ep: 9545 | Ep_r: 7
w3 Ep: 9546 | Ep_r: 7
w2 Ep: 9548 | Ep_r: 7
w0 Ep: 9548 | Ep_r: 7
w1 Ep: 9549 | Ep_r: 7
w2 Ep: 955

w2 Ep: 9878 | Ep_r: 7
w3 Ep: 9879 | Ep_r: 7
w2 Ep: 9880 | Ep_r: 7
w0 Ep: 9882 | Ep_r: 7
w1 Ep: 9881 | Ep_r: 7
w3 Ep: 9883 | Ep_r: 7
w2 Ep: 9884 | Ep_r: 7
w1 Ep: 9885 | Ep_r: 7
w0 Ep: 9887 | Ep_r: 7
w3 Ep: 9886 | Ep_r: 7
w2 Ep: 9888 | Ep_r: 7
w1 Ep: 9889 | Ep_r: 7
w3 Ep: 9890 | Ep_r: 7
w1 Ep: 9891 | Ep_r: 7
w0 Ep: 9893 | Ep_r: 7
w2 Ep: 9892 | Ep_r: 7
w1 Ep: 9894 | Ep_r: 7
w2 Ep: 9895 | Ep_r: 7
w0 Ep: 9897 | Ep_r: 7
w3 Ep: 9897 | Ep_r: 7
w1 Ep: 9898 | Ep_r: 7
w3 Ep: 9899 | Ep_r: 7
w2 Ep: 9900 | Ep_r: 7
w0 Ep: 9902 | Ep_r: 7
w3 Ep: 9901 | Ep_r: 7
w1 Ep: 9903 | Ep_r: 7
w2 Ep: 9905 | Ep_r: 7
w3 Ep: 9905 | Ep_r: 7
w1 Ep: 9906 | Ep_r: 7
w2 Ep: 9908 | Ep_r: 7
w3 Ep: 9907 | Ep_r: 7
w3 Ep: 9909 | Ep_r: 7
w1 Ep: 9910 | Ep_r: 7
w2 Ep: 9911 | Ep_r: 7
w3 Ep: 9912 | Ep_r: 7
w1 Ep: 9913 | Ep_r: 7
w2 Ep: 9914 | Ep_r: 7
w1 Ep: 9915 | Ep_r: 7
w3 Ep: 9916 | Ep_r: 7
w2 Ep: 9917 | Ep_r: 7
w1 Ep: 9918 | Ep_r: 7
w3 Ep: 9919 | Ep_r: 7
w2 Ep: 9920 | Ep_r: 7
w3 Ep: 9922 | Ep_r: 7
w0 Ep: 9922 | Ep_r: 7
w1 Ep: 992

In [ ]:
    import matplotlib.pyplot as plt
    plt.plot(res)
    plt.ylabel('Moving average ep reward')
    plt.xlabel('Step')
    plt.show()